# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [3]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


In [4]:
# Display loading data
adult_dt

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [5]:
# Import necessary libraries and modules for preprocessing and modeling
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression

# Separate features (X) and target variable (Y) from the dataset
X = adult_dt.drop('income', axis=1) # Features (all columns except 'income')
Y = adult_dt['income'] # Target variable ('income')

# Split the dataset into training and testing sets
# 70% of the data will be used for training and 30% for testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


In [7]:
# Display the column names of the X DataFrame
X.columns


Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'],
      dtype='object')

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

- The random_state of the splitting function, such as train_test_split from scikit-learn, is a parameter that controls the randomness of the data shuffling and splitting process. When you specify a random_state value, you essentially set a seed for the random number generator used in the splitting process. This ensures that the split of the data into training and testing sets is consistent and reproducible each time the code is executed with the same random_state.

For example, using random_state=42 in the train_test_split function guarantees that the resulting training and testing sets will be the same every time you run the code, as long as the input data remains unchanged.

- Why is random_state useful?

    - Reproducibility: By setting a random_state, you allow others to exactly replicate your results, which is critical in scientific research and data analysis.
    - Consistency: A consistent train-test split allows you to compare different models or parameter settings on the same data partition, ensuring that performance differences are due to the models themselves rather than different data splits.
    - Debugging: A fixed random_state makes it easier to trace and reproduce specific issues or behaviors when debugging code or verifying results.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [8]:
# Import necessary libraries and modules for preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# Identify numeric features in the dataset
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
# Create a pipeline for preprocessing numeric features
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

# Identify categorical features in the dataset
categorical_features = X.select_dtypes(include=['object']).columns
# Create a pipeline for preprocessing categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Impute missing values using the most frequent value
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary')) # Encode categorical variables using one-hot encoding
])

# Combine the numeric and categorical transformers into a single preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', numeric_transformer, numeric_features),
        ('cat_transforms', categorical_transformer, categorical_features)
    ])

# Create a pipeline that applies the preprocessor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)]) # Apply the preprocessor to the data

In [9]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object'))]))])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [10]:
# Import necessary libraries and modules for preprocessing and modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Create a pipeline that includes preprocessing and the RandomForest classifier
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor), # Apply the preprocessor to the data
    ('classifier', RandomForestClassifier()) # Apply the RandomForestClassifier to the preprocessed data
])

In [11]:
pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object'))])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [12]:
# Import necessary libraries and modules for evaluation and cross-validation
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score
from sklearn.model_selection import GridSearchCV
import warnings

# Ignore warnings for cleaner output
warnings.filterwarnings('ignore')

# Define a dictionary of scoring metrics for evaluation
scoring = {
    'neg_log_loss': 'neg_log_loss', # Negative log loss
    'roc_auc': 'roc_auc', # ROC AUC score
    'accuracy': 'accuracy', # Accuracy score
    'balanced_accuracy': make_scorer(balanced_accuracy_score) # Balanced accuracy score
}

# Define the parameter grid for hyperparameter tuning of the RandomForestClassifier
param_grid = {
    'classifier__n_estimators': [10, 50, 100],
    'classifier__max_depth': [None, 10, 20, 30]
}

# Initialize GridSearchCV with the pipeline, parameter grid, 5-fold cross-validation, and accuracy as the primary scoring metric
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the training data
grid_search.fit(X_train, Y_train)

# Perform cross-validation with the best estimator found by grid search and evaluate using the defined scoring metrics
cv_results = cross_validate(
    grid_search.best_estimator_, # Best model found by grid search
    X_train, # Training features
    Y_train,  # Training target variable
    cv=5, # 5-fold cross-validation
    scoring=scoring, # Scoring metrics to evaluate
    return_train_score=True # Return training scores as well
)

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [13]:
# Convert the cross-validation results to a pandas DataFrame
cv_df = pd.DataFrame(cv_results)

# Sort the DataFrame by the 'test_neg_log_loss' column in ascending order
cv_df_sorted = cv_df.sort_values(by='test_neg_log_loss')

# Display the first few rows of the sorted DataFrame
cv_df_sorted.head()

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy
1,10.174149,0.249332,-0.308909,-0.191862,0.913855,0.984342,0.855012,0.931608,0.761095,0.874348
0,9.147463,0.227391,-0.307644,-0.189409,0.914471,0.984887,0.858083,0.932759,0.770496,0.880029
2,9.246559,0.217445,-0.307450,-0.191375,0.915135,0.984503,0.860026,0.930569,0.767927,0.876223
4,9.155713,0.221375,-0.305308,-0.191134,0.915967,0.984562,0.863537,0.934573,0.776090,0.885093
3,9.630366,0.336099,-0.298937,-0.192748,0.920300,0.984302,0.868144,0.930295,0.786823,0.877272


Calculate the mean of each metric. 

In [14]:
# Calculate the mean of training metrics from the cross-validation results
train_metrics = cv_df.filter(like='train').mean().to_dict()

# Calculate the mean of validation metrics from the cross-validation results
val_metrics = cv_df.filter(like='test').mean().to_dict()

# Print the mean training metrics
print("Training Metrics Mean: ", train_metrics)

# Print the mean validation metrics
print("Validation Metrics Mean: ", val_metrics)

Training Metrics Mean:  {'train_neg_log_loss': -0.19130536351940652, 'train_roc_auc': 0.9845194459049817, 'train_accuracy': 0.9319607805945752, 'train_balanced_accuracy': 0.8785931430611518}
Validation Metrics Mean:  {'test_neg_log_loss': -0.3056498227687091, 'test_roc_auc': 0.9159455174102478, 'test_accuracy': 0.8609603731910063, 'test_balanced_accuracy': 0.7724859576976516}


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [15]:
# Retrieve the best estimator from the grid search
best_estimator = grid_search.best_estimator_
# Fit the best estimator on the training data
best_estimator.fit(X_train, Y_train)

# Predict probabilities for the test set
Y_pred_proba = best_estimator.predict_proba(X_test)
# Predict class labels for the test set
Y_pred = best_estimator.predict(X_test)

# Calculate performance metrics for the test set
test_metrics = {
    'neg_log_loss': log_loss(Y_test, Y_pred_proba), # Calculate negative log loss
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]), # Calculate ROC AUC score
    'accuracy': accuracy_score(Y_test, Y_pred), # Calculate accuracy
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred) # Calculate balanced accuracy
}

# Print the test set performance metrics
print("Test Set Performance Metrics: ", test_metrics)

Test Set Performance Metrics:  {'neg_log_loss': 0.3062047856718444, 'roc_auc': np.float64(0.9134523650111559), 'accuracy': 0.8620124884839799, 'balanced_accuracy': np.float64(0.7680342498668183)}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

Loading the data and immediately recoding the target variable income in the first code chunk is convenient for several reasons:

- Binary Target Transformation: 
- By recoding the income variable to a binary format (0 and 1), you simplify the target variable for binary classification tasks. This is particularly useful as many machine learning algorithms in scikit-learn expect the target variable to be in a binary format for classification problems.

- Consistency: 
- Performing the transformation right after loading the data ensures that the target variable is consistently formatted throughout your analysis, reducing the chance of errors or inconsistencies later in your workflow.

- Simplified Evaluation: 
- Scikit-learn's model evaluation tools and metrics, such as accuracy, precision, recall, and ROC-AUC, are designed to work seamlessly with binary target variables. By converting the income variable to a binary format, you can directly apply these evaluation methods without additional preprocessing.

- Streamlined Workflow: 
- Including the transformation step in the data loading process makes the code more concise and readable. It integrates data preparation into a single step, which is easier to manage and understand.

The specific line of code:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```
- Data Loading: The pd.read_csv function reads the dataset into a pandas DataFrame.

- Column Assignment: The header=None and names=columns parameters ensure that the columns are correctly named according to the provided list columns.

- Income Recoding: The assign method with a lambda function strips any whitespace from the income values, checks if the income is greater than 50K, and converts this boolean result to an integer (1 for True, 0 for False).

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.